In [50]:
import json
import numpy as np
import pandas as pd

data_file  = "../data/cleaned_data.csv"

In [51]:
# read true data
f = open(data_file,"r")
x = f.readlines()
f.close()

X = []
Y = []
for i in range(1, len(x)):
    tem = x[i].strip().split(",")
    X.append([float(j) for j in tem[1:-1]])
    Y.append(int(tem[-1]))
X = np.array(X)
Y = np.array(Y)

nlabel = 4
d1, d2, d3 = 180, 18, 42

# training-test split
indexes = [[1, 15, 13, 12,  8, 25, 28, 50, 41, 35, 34, 54],
[16,  3, 18, 19, 21, 30, 24 ,51, 47, 46, 33, 55],
[17,  0, 11 , 6 ,29 ,23 ,44 ,39 ,43 ,40, 53],
[10,  4,  9 , 2 ,26 ,27 ,38 ,36, 32, 42 ,52],
[5 ,22 ,20, 14 , 7 ,31 ,45, 37, 48, 49, 56]]

# supervised = {}
# unsupervised = {}
# for i in indexes:
#     print(Y[i])

[0 0 0 0 0 1 1 2 2 2 2 3]
[0 0 0 0 0 1 1 2 2 2 2 3]
[0 0 0 0 1 1 2 2 2 2 3]
[0 0 0 0 1 1 2 2 2 2 3]
[0 0 0 0 0 1 2 2 2 2 3]


In [55]:
supervised = {}
for k0 in range(1, 16):
    k1 = k0
    T = []
    for i in range(nlabel):
        tem = np.block([
            [np.zeros((k0*i,k0+k1))],
            [np.eye(k0), np.zeros((k0, k1))],
            [np.zeros((k0*(nlabel-i-1),k0+k1))],
            [np.zeros((k1,k0)), np.eye(k1)]
        ])
        T.append(tem)

    # read the result
    cur= {}
    start_file = (k0 - 1) * 5 + 1
    for i in range(0,5):
        with open("./supervised/MOFA{}.txt".format(str(start_file+i))) as f:
            x = f.read()
        cur[i] = json.loads(x)
        # cur[i]["accuracy"] = sum(cur[i]["prediction"]["Y"] == Y[indexes[i]])/len(indexes[i])
    supervised[k0] = cur


unsupervised = {}
for k0 in range(1, 16):
    k1 = k0

    # read the result
    cur= {}
    start_file = (k0 - 1) * 5 + 1
    for i in range(0,5):
        with open("./unsupervised/MOFA_un{}.txt".format(str(start_file+i))) as f:
            x = f.read()
        cur[i] = json.loads(x)
        # cur[i]["accuracy"] = sum(cur[i]["prediction"]["Y"] == Y[indexes[i]])/len(indexes[i])
    unsupervised[k0] = cur

res = {"supervised": supervised, "unsupervised": unsupervised}
with open("res.txt", "w") as f:
    f.write(json.dumps(res))


In [ ]:
with open("res.txt", "r") as f:
    x = f.read()
res = json.loads(x)

In [71]:
for k0 in range(1, 16):
    k1 = k0
    T = []
    for i in range(nlabel):
        tem = np.block([
            [np.zeros((k0*i,k0+k1))],
            [np.eye(k0), np.zeros((k0, k1))],
            [np.zeros((k0*(nlabel-i-1),k0+k1))],
            [np.zeros((k1,k0)), np.eye(k1)]
        ])
        T.append(tem)

    # similate x from estimate
    sim_N = 30
    
    sim_x = np.array([list(range(241))]) # len(each row) = 241
    cur = res["supervised"][k0]

    for i in range(0,5):
        estimate = cur[i]["estimation"]
        Mu, Lambda = [estimate["Mu1"], estimate["Mu2"], estimate["Mu3"]], [estimate["Lambda1"], estimate["Lambda2"], estimate["Lambda3"]]
        Mu, Lambda = [np.array(mus) for mus in Mu], [np.array(lambdas) for lambdas in Lambda]
        Mu = np.concatenate((Mu[0], Mu[1], Mu[2]), axis=1)
        Lambda = np.concatenate((Lambda[0], Lambda[1], Lambda[2]), axis=1)
        Tau = Mu * Lambda

        G = np.array(cur[i]["prediction"]["G"])
        Ybar = cur[i]["prediction"]["Y"]
        # Ybar = Y[indexes[i]]
        U = np.array([np.dot(T[Ybar[i]], G[i]) for i in range(len(Ybar))])

        for j in range(len(indexes[i])):
            u = U[j]
            tem = np.zeros((sim_N, 241)) + indexes[i][j]
            for k in range(0,240):
                lambdax = np.dot(u, Lambda[:, k])
                taux = np.dot(u, Tau[:, k])
                sx = 1/lambdax
                mux = sx*taux
                tem[:,k] = np.random.normal(mux, np.sqrt(sx),sim_N)
            sim_x = np.concatenate((sim_x, tem), axis = 0)
    np.savetxt("./simulated/sim_x_{}.csv".format(str(k0)), sim_x[1:,:], delimiter=",",fmt="%.2f")

    sim_x = np.array([list(range(241))]) # len(each row) = 241
    cur = res["unsupervised"][k0]

    for i in range(0,5):
        estimate = cur[i]["estimation"]
        Mu, Lambda = [estimate["Mu1"], estimate["Mu2"], estimate["Mu3"]], [estimate["Lambda1"], estimate["Lambda2"], estimate["Lambda3"]]
        Mu, Lambda = [np.array(mus) for mus in Mu], [np.array(lambdas) for lambdas in Lambda]
        Mu = np.concatenate((Mu[0], Mu[1], Mu[2]), axis=1)
        Lambda = np.concatenate((Lambda[0], Lambda[1], Lambda[2]), axis=1)
        Tau = Mu * Lambda

        U = np.array(cur[i]["prediction"]["U"])
        Ybar = cur[i]["prediction"]["Y"]
        # Ybar = Y[indexes[i]]
        # U = np.array([np.dot(T[Ybar[i]], G[i]) for i in range(len(Ybar))])

        for j in range(len(indexes[i])):
            u = U[j]
            tem = np.zeros((sim_N, 241)) + indexes[i][j]
            for k in range(0,240):
                lambdax = np.dot(u, Lambda[:, k])
                taux = np.dot(u, Tau[:, k])
                sx = 1/lambdax
                mux = sx*taux
                tem[:,k] = np.random.normal(mux, np.sqrt(sx),sim_N)
            sim_x = np.concatenate((sim_x, tem), axis = 0)
    np.savetxt("./simulated/sim_x_un_{}.csv".format(str(k0)), sim_x[1:,:], delimiter=",",fmt="%.2f")
    

In [68]:
accuracy = {"supervised": [], "unsupervised": []}
for meth in ["supervised", "unsupervised"]:
    for i in range(1,16):
        tem = 0
        for j in range(len(indexes)):
           
            tem += sum(res[meth][i][j]["prediction"]["Y"] == Y[indexes[j]])
        accuracy[meth].append(tem/57)


In [70]:
print(accuracy["supervised"])
print(accuracy["unsupervised"])

[0.6140350877192983, 0.49122807017543857, 0.45614035087719296, 0.49122807017543857, 0.5087719298245614, 0.49122807017543857, 0.47368421052631576, 0.49122807017543857, 0.543859649122807, 0.5789473684210527, 0.5614035087719298, 0.5087719298245614, 0.6491228070175439, 0.5964912280701754, 0.5614035087719298]
[0.22807017543859648, 0.17543859649122806, 0.2982456140350877, 0.2631578947368421, 0.14035087719298245, 0.17543859649122806, 0.17543859649122806, 0.24561403508771928, 0.19298245614035087, 0.22807017543859648, 0.14035087719298245, 0.2631578947368421, 0.2807017543859649, 0.2631578947368421, 0.3333333333333333]


In [76]:
k0 = 3
nt = 5*k0
estimate = res["supervised"][k0][1]["estimation"]
Mu, Lambda = [estimate["Mu1"], estimate["Mu2"], estimate["Mu3"]], [estimate["Lambda1"], estimate["Lambda2"], estimate["Lambda3"]]
Mu, Lambda = [np.array(mus) for mus in Mu], [np.array(lambdas) for lambdas in Lambda]
Mu = np.concatenate((Mu[0], Mu[1], Mu[2]), axis=1)
Lambda = np.concatenate((Lambda[0], Lambda[1], Lambda[2]), axis=1)
Tau = Mu * Lambda
S = 1/Lambda
gene = []
def dist(t1, t2, i):
    mu1, mu2, s1, s2 = Mu[t1,1], Mu[t1,i], S[t1,i], S[t2,i]
    s = (s1+s2)/2
    return (mu1-mu2)**2/s/8+1/2*np.log(s/np.sqrt(s1*s2))

for i in range(240):
    tem = []
    for t1 in range(nt-1):
        for t2 in range (t1+1,nt):
            tem.append(dist(t1,t2,i))
    gene.append(max(tem))



In [78]:
gene1 = gene

In [79]:
k0 = 9
nt = 5*k0
estimate = res["supervised"][k0][1]["estimation"]
Mu, Lambda = [estimate["Mu1"], estimate["Mu2"], estimate["Mu3"]], [estimate["Lambda1"], estimate["Lambda2"], estimate["Lambda3"]]
Mu, Lambda = [np.array(mus) for mus in Mu], [np.array(lambdas) for lambdas in Lambda]
Mu = np.concatenate((Mu[0], Mu[1], Mu[2]), axis=1)
Lambda = np.concatenate((Lambda[0], Lambda[1], Lambda[2]), axis=1)
Tau = Mu * Lambda
S = 1/Lambda
gene = []
def dist(t1, t2, i):
    mu1, mu2, s1, s2 = Mu[t1,1], Mu[t1,i], S[t1,i], S[t2,i]
    s = (s1+s2)/2
    return (mu1-mu2)**2/s/8+1/2*np.log(s/np.sqrt(s1*s2))

for i in range(240):
    tem = []
    for t1 in range(nt-1):
        for t2 in range (t1+1,nt):
            tem.append(dist(t1,t2,i))
    gene.append(max(tem))



In [87]:
gene = np.array(gene)
n = 20
idx = np.argpartition(gene, -n)[-n:]
indices = idx[np.argsort((-gene)[idx])]
indices

array([217, 202, 205, 231, 200, 198, 199, 224, 209, 207, 211, 204, 233,
       235, 218, 234, 230, 223, 201, 206])

In [90]:
print(gene[indices.tolist()])
print(gene[list(range(10))])


[7.53055417 6.84389654 6.28953733 5.78788215 5.46889616 4.92995231
 4.51699618 4.42768504 4.11941815 3.99834579 3.60219404 3.34480437
 3.26045544 3.12172397 2.9064809  2.66039241 2.46078044 2.40228011
 2.26054697 2.20225871]
[0.74039333 0.03482271 0.46393796 0.40341546 0.26484168 0.53092709
 0.59888151 0.80673752 0.80248388 0.98687181]
